In [1]:
import numpy as np
import itertools
import setuptools.dist
from deepface import DeepFace
import pandas as pd

### - detector 'opencv', model 'Facenet512'

In [2]:
def get_embedding(photo_path):
    return DeepFace.represent(
                    photo_path, 
                    model_name="Facenet512", 
                )[0]["embedding"]

Funkcje do porównywania wektorów reprezentacji(embeding).

In [3]:
def cosine_similarity(embedding1, embedding2):
    dot_product = np.dot(embedding1, embedding2)
    norm1 = np.linalg.norm(embedding1)
    norm2 = np.linalg.norm(embedding2)
    return dot_product / (norm1 * norm2)

def euclidean_distance(embedding1, embedding2):
    return np.linalg.norm(embedding1 - embedding2)

In [45]:
def photo_comparision(photo_list: list[str], file_name):
    rows = {"photo 1": [], "photo 2": [], "cosine similarity": [], "euclidean_distance": []}
    i=0
    photo_embd = {}
    for photo in photo_list:
        photo_embd[photo] = np.array(get_embedding(photo))
        
    pairs = itertools.combinations(photo_list, 2)
    for photo1, photo2 in pairs:
        rows["photo 1"].append(photo1.split("/")[-1])
        rows["photo 2"].append(photo2.split("/")[-1])
        rows["cosine similarity"].append(cosine_similarity(photo_embd[photo1], photo_embd[photo2]))
        rows["euclidean_distance"].append(euclidean_distance(photo_embd[photo1], photo_embd[photo2]))

    df = pd.DataFrame(rows)
    print(df)
    df.to_csv(file_name, mode='w')

In [18]:
def all_to_one_comparision(model_photo, photo_list, file_name):
    rows = {"photo 1": [], "photo 2": [], "cosine similarity": [], "euclidean_distance": []}
    i=0
    photo_embd = {}
    pairs = []
    for photo in photo_list:
        photo_embd[photo] = np.array(get_embedding(photo))

    for photo in photo_list: 
        pairs.append([photo, model_photo])

    for photo1, photo2 in pairs:
        rows["photo 1"].append(photo1.split("/")[-1])
        rows["photo 2"].append(photo2.split("/")[-1])
        rows["cosine similarity"].append(cosine_similarity(photo_embd[photo1], photo_embd[photo2]))
        rows["euclidean_distance"].append(euclidean_distance(photo_embd[photo1], photo_embd[photo2]))

    df = pd.DataFrame(rows)
    print(df)
    df.to_csv(file_name, mode='a')

### Część 0 - materiały do częśli teorytycznej

In [50]:
embedding = get_embedding("photos/part_2/9.jpg")
print(embedding)
print(len(embedding))

[-0.7090968489646912, 2.089310884475708, 2.173694610595703, 0.4953502416610718, 0.11321710050106049, 1.4992707967758179, -1.1296100616455078, -0.9756104350090027, 0.15121683478355408, -3.1171023845672607, -0.5459817051887512, -2.0308101177215576, 0.6250482201576233, 0.7259276509284973, -1.2008994817733765, -0.9043356776237488, 1.1271367073059082, 2.552112340927124, 0.14744162559509277, -0.897793173789978, -1.3285459280014038, 0.0839587077498436, 2.346958637237549, 0.20037807524204254, 0.07221127301454544, 0.5501185059547424, 1.3213329315185547, 0.07597164064645767, 0.8175913691520691, 1.2459381818771362, 0.1689722090959549, 0.7968223094940186, -0.8102869987487793, -0.9576781392097473, 1.443926453590393, 0.9809365272521973, 1.5571839809417725, -0.925899863243103, 0.4616210460662842, 0.7984659075737, 0.15844325721263885, -0.2961718440055847, 1.2389670610427856, -0.7472403049468994, 0.8960867524147034, -1.9443715810775757, -1.282579779624939, 1.6980050802230835, -0.8407256603240967, -0.43

### Część 1 - porównywanie zdjęć różnych osób.

In [19]:
photos = ["photos/part_1/photo-1.jpeg", "photos/part_1/photo-2.jpeg", "photos/part_1/photo-3.jpeg"]
photo_comparision(photos, "./results/part_1")

        photo 1       photo 2  cosine similarity  euclidean_distance
0  photo-1.jpeg  photo-2.jpeg           0.050765           31.856541
1  photo-1.jpeg  photo-3.jpeg          -0.030123           34.245827
2  photo-2.jpeg  photo-3.jpeg           0.399200           24.853003


### Część 2 - porównywanie różnych zdjęć tej samej osoby.

In [37]:
import cv2
import matplotlib.pyplot as plt
img = cv2.imread("photos/part_2/10.jpg")

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [39]:
img = img[...,::-1]

In [ ]:
img_blurred = cv2.GaussianBlur(img, (5,5), 0)
plt.imshow(img_blurred)

In [47]:
photos = ["photos/part_2/3.jpg", "photos/part_2/4.jpg", "photos/part_2/9.jpg", "photos/part_2/10.jpg", "photos/part_2/10_blurred.jpg", "photos/part_2/11.jpg"]
all_to_one_comparision("photos/part_2/4.jpg", photos, "./results/part_2")

          photo 1 photo 2  cosine similarity  euclidean_distance
0           3.jpg   4.jpg           0.688173           18.432031
1           4.jpg   4.jpg           1.000000            0.000000
2           9.jpg   4.jpg           0.590857           21.199532
3          10.jpg   4.jpg           0.129858           30.202699
4  10_blurred.jpg   4.jpg           0.232437           29.189701
5          11.jpg   4.jpg           0.618557           20.903867


### Tests

In [56]:
photos = ["photos/part_2/10.jpg","photos/part_2/10_blurred.jpg", "photos/part_2/4.jpg"]
all_to_one_comparision("photos/part_2/4.jpg", photos, "./results/tests")
photos = ["photos/tests/4.jpg","photos/tests/4_bw.jpg"]
all_to_one_comparision("photos/tests/4.jpg", photos, "./results/tests")

          photo 1 photo 2  cosine similarity  euclidean_distance
0          10.jpg   4.jpg           0.129858           30.202699
1  10_blurred.jpg   4.jpg           0.232437           29.189701
2           4.jpg   4.jpg           1.000000            0.000000
    photo 1 photo 2  cosine similarity  euclidean_distance
0     4.jpg   4.jpg           1.000000            0.000000
1  4_bw.jpg   4.jpg           0.975955            5.033678
